In [36]:

import scipy.io.wavfile as wavfile
from scipy.linalg import dft
import numpy as np
import matplotlib.pyplot as plt

In [37]:
# DFT of image rotated by 90 degrees.
def get_DFT_mat(v, inverse=False):
    v_np = np.array(v)
    DFT_size = v.shape[1]
    DFT_mat = np.empty(shape=(DFT_size,DFT_size), dtype=complex)
    conjugate = 1 if inverse==False else -1
    w = np.power(np.e, (conjugate*2j)*np.pi/DFT_size, dtype=complex)
    for i in range(DFT_size):
        for j in range(DFT_size):
            DFT_mat[i,j] = np.power(w, i*j, dtype=complex)
    return (1/np.sqrt(DFT_size)) * np.transpose(DFT_mat @ np.transpose(v_np))

In [38]:
ncol=512
sky_vec = wavfile.read("./skycastle.wav")[1]
sky_vec_dist = wavfile.read("./skycastle-distortion.wav")[1]
skycastel_orig_mat = np.reshape(sky_vec, (sky_vec.size//ncol,ncol))
skycastel_dist_mat = np.reshape(sky_vec_dist, (sky_vec_dist.size//ncol,ncol))

A = get_DFT_mat(skycastel_orig_mat)
B = get_DFT_mat(skycastel_dist_mat)

In [39]:
#checking A is invertible
u, s, v = np.linalg.svd(A)
print(np.amin(s),np.amax(s))

10.281968513289344 801072.1954419574


In [40]:
#our func map
C = np.linalg.pinv(A) @ B


In [41]:
#checking C is invertible
u, s, v = np.linalg.svd(C)
print(np.amin(s),np.amax(s))

0.0001420272924388475 10.932299570531166


In [42]:
dist_to_fix_vec = wavfile.read("./totoro-distortion.wav")[1]
dist_to_fix_mat = np.reshape(dist_to_fix_vec, (dist_to_fix_vec.size//ncol,ncol))
fourier_totoro = get_DFT_mat(dist_to_fix_mat) @ np.linalg.pinv(C)
reconst_totoro = np.reshape(np.real(get_DFT_mat(fourier_totoro, inverse=True)), -1)
reconst_skycastle = np.reshape(np.real(get_DFT_mat(B @ np.linalg.pinv(C), inverse=True)), -1)
orig_totoro = wavfile.read("./totoro.wav")

In [43]:
wavfile.write("totoro-reconst.wav",orig_totoro[0],reconst_totoro.astype(np.dtype('i2')))
wavfile.write("skycastle-reconst.wav",wavfile.read("./skycastle.wav")[0],reconst_skycastle.astype(np.dtype('i2')))